In [ ]:
# default_exp tree

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import re
import IPython, graphviz

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import export_graphviz
from copy import deepcopy

from tree_evasion.core import *

SEED = 41
np.random.seed(SEED)

In [ ]:
#export
_all_ = ['Tree']

# Tree

> How to represent a trained Random Forest Classifier ( scikit-learn ) model?

In [ ]:
#export
class Tree:
    def __init__(self, tree):
        self.tree   = tree
        
    def is_leaf(self, index):
        return self.tree.feature[index] < 0
    
    def get_left_child_index(self, index):
        return self.tree.children_left[index]
        
    def get_right_child_index(self, index):
        return self.tree.children_right[index]
        
    def predicate(self, index):
        return (self.tree.feature[index], self.tree.threshold[index])
    
    def prediction(self, index):
        class_prob = self.tree.value[index].ravel()
        class_prob_sum = np.sum(class_prob)
        return class_prob[1] / class_prob_sum
    
    def constraints(self, x):
        instance = x.reshape(1, -1).astype(np.float32)
        path = self.tree.decision_path(instance).toarray().ravel()
        nodes = np.where(path == 1)[0][:-1]
        
        c = {}
        
        for nidx in nodes:
            fnode = self.tree.feature[nidx]
            
            instance_feat = instance.ravel()[fnode]
            
            if instance_feat <= self.tree.threshold[nidx]:
                if fnode not in c:
                    c[fnode] = [-np.inf, self.tree.threshold[nidx]]
                else:
                    if self.tree.threshold[nidx] < c[fnode][1]:
                        c[fnode][1] = self.tree.threshold[nidx]
            else:
                if fnode not in c:
                    c[fnode] = [self.tree.threshold[nidx], np.inf]
                else:
                    if self.tree.threshold[nidx] > c[fnode][0]:
                        c[fnode][0] = self.tree.threshold[nidx]
                    
        return c

### Export

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()